# Language Model and Application for Spelling Error Correction
## Objective: Develop a simple English syntax error correction program

## Home Exercise:

a) Improve the model by using interpolation smoothing with the "Stupid Backoff" method (Brants et al., 2007).

b) Compare with the results from In Class Exercise.

c) Use the newly built model to generate the next words for a given word sequence.

d) Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position. (from difflib import get_close_matches)

## Import Libraries

In [1]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.7 MB/s eta 0:00:00


In [2]:
import string
import re
import nltk
nltk.download('punkt_tab')
import contractions
from collections import Counter
from difflib import get_close_matches

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Data Downloading

In [3]:
with open('/content/tedtalk.txt') as file:
  docs = file.read()

## Data Preprocessing

In [4]:
vocab = set()     # Number of unique word
token_count = 0   # Number of token

# Lower case character
def text_lowercase(text):
    text = text.lower()
    return text

# Split into sentences
def sent_tokenize(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

# Removing contractions and Keep number and letters
def remove_punctuation(text):
    text = contractions.fix(text)
    text = re.sub(r'[^a-z0-9\' ]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

# Tokenize the text into words
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

# Data preprocessing
def preprocess_text(text, is_addvocab=False):
    global token_count
    text = text_lowercase(text)
    sentences = sent_tokenize(text)
    processed_sentences = []
    for sentence in sentences:
      sentence = remove_punctuation(sentence)
      sentence = tokenize(sentence)
      if is_addvocab:
        token_count += len(sentence)
        for token in sentence:
          vocab.add(token)
      processed_sentences.append(sentence)

    return processed_sentences

In [5]:
preprocess_data = preprocess_text(docs,True)
vocab.add('<s>')
vocab.add('</s>')
print("Number of unique words:", len(vocab))
print("Number of tokens:", token_count)

Number of unique words: 68782
Number of tokens: 7396888


## a) Improve the model by using interpolation smoothing with the "Stupid Backoff" method (Brants et al., 2007).

In [6]:
def correct_word(word, cutoff=0.6):
    matches = get_close_matches(word, vocab, n=5, cutoff=0.7)
    return matches[0] if matches else word


In [15]:
class NgramModel:
    def __init__(self, n, v, token_count):
      self.n = n
      self.ngrams = Counter()
      self.context = Counter()
      self.vocab = v
      self.vocab_size = len(v)
      self.token_count = token_count
      self.discount_factor = 0.4
      # if self.n == 1:
      #   self.ngrams[(tuple(['<s>']))] += 1
      #   self.ngrams[(tuple(['</s>']))] += 1


    def padding_both_ends(self, tokens):
      if self.n == 1:
        return tokens
      else:
        return ['<s>'] * (self.n - 1) + tokens + ['</s>'] * (self.n - 1) # adding <s> <s> and </s> </s> for trigram


    # Train the model by counting n-grams and contexts
    def train(self, corpus):
      for tokens in corpus:
        tokens = self.padding_both_ends(tokens)
        for i in range(len(tokens) - self.n + 1):
          ngram = tuple(tokens[i:i+self.n]) # Create n-gram
          self.ngrams[ngram] += 1
          self.context[ngram[:-1]] += 1


    # Calculate the probability of an n-gram using Laplace smoothing
    def laplace_smoothing(self, ngram):
      if self.n != 1:
        return (self.ngrams[ngram] + 1) / (self.context[ngram[:-1]] + self.vocab_size)
      else:
        return (self.ngrams[ngram] + 1) / (self.token_count + self.vocab_size)


    # Calculate the probability of an n-gram using Stupid backoff
    def stupid_backoff(self, ngram):
      for k in range(self.n, 0, -1):
          backoff_ngram = ngram[-k:]
          if self.ngrams[backoff_ngram] > 0:
            if len(backoff_ngram) != 1:
              prob = self.ngrams[backoff_ngram] / self.context[backoff_ngram[:-1]]
              return prob
            else:
              # print(backoff_ngram, 'here')
              prob = self.ngrams[backoff_ngram] / self.token_count
              return prob
      if len(ngram) > 1:
          # If a higher-order n-gram has a zero count, we simply backoff to a lower order n-gram
          return self.discount_factor * self.stupid_backoff(ngram[1:])
      return 1 / (self.token_count + self.vocab_size) # Return a small probability for UNK


    # Calculate the probability of a given sentence
    def sentence_probability(self, sentence):
      laplace_prob = 1
      backoff_prob = 1
      tokens = preprocess_text(sentence)[0]
      tokens = self.padding_both_ends(tokens)
      for i in range(len(tokens) - self.n + 1):
          ngram = tuple(tokens[i:i+self.n])

          laplace = self.laplace_smoothing(ngram)
          backoff = self.stupid_backoff(ngram)

          laplace_prob *= laplace
          backoff_prob *= backoff
      return laplace_prob, backoff_prob


    # Calculate the perplexity of a given sentence
    def sentence_perplexity(self, sentence):
      tokens = preprocess_text(sentence)[0]
      tokens = self.padding_both_ends(tokens)
      N = len(tokens)
      laplace_prob, backoff_prob = self.sentence_probability(sentence)
      return laplace_prob ** (-1 / N), backoff_prob ** (-1 / N)


    # Generate a next word for a given word sequence.
    def generate_next_word(self,sentence):
      tokens = preprocess_text(sentence)[0]

      max_prob = 0
      best_word = None

      print(f"Given Sequence: {sentence}")
      print(f"Candidate Words and Probabilities:")

      for word in self.vocab:
        if word == '<s>':
          continue

        # Create a candidate bgram and select the one with highest probability as predicted netxt word
        candidate_ngram = tuple(tokens[-(self.n - 1):] + [word])
        prob = self.stupid_backoff(candidate_ngram)

        if prob > max_prob:
            max_prob = prob
            best_word = word

      print(f"\nBest Next Word: {best_word} (Probability: {max_prob})")
      return best_word


    # Generate a full sentence from initial sentence
    def generate_sentence(self, initial_sequence, max_length=10):
      tokens = preprocess_text(initial_sequence)[0]

      print(f"Initial Sequence: {' '.join(tokens)}\nGenerating...\n")

      for _ in range(max_length):
          next_word = self.generate_next_word(' '.join(tokens[-(self.n - 1):]))

          if next_word =='</s>':
              break  # End the sentence if the predicted word is </s>

          tokens.append(next_word)

          print(f"Current Sentence: {' '.join(tokens)}")

      generated_sentence = ' '.join(tokens)
      print(f"\nGenerated Sentence: {generated_sentence}")
      return generated_sentence


     # Generate a full sentence from misspelled initial sentence
    def generate_sentence_for_misspelled(self, initial_sequence, max_length=10):
      tokens = preprocess_text(initial_sequence)[0]
      for i in range(len(tokens)):
        corrected_word = correct_word(word=tokens[i])
        if corrected_word != tokens[i]:
            tokens[i] = corrected_word

      print(f"Initial Sequence: {' '.join(tokens)}\nGenerating...\n")

      for _ in range(max_length):
          next_word = self.generate_next_word(' '.join(tokens[-(self.n - 1):]))

          if next_word =='</s>':
              break  # End the sentence if the predicted word is </s>

          tokens.append(next_word)

          print(f"Current Sentence: {' '.join(tokens)}")

      generated_sentence = ' '.join(tokens)
      print(f"\nGenerated Sentence: {generated_sentence}")
      return generated_sentence


In [16]:
# Train unigram, bigram, and trigram model
unigram_model = NgramModel(n=1, v=vocab, token_count=token_count)
bigram_model = NgramModel(n=2, v=vocab, token_count=token_count)
trigram_model = NgramModel(n=3, v=vocab, token_count=token_count)

unigram_model.train(preprocess_data)
bigram_model.train(preprocess_data)
trigram_model.train(preprocess_data)

In [17]:
bigram_model.ngrams.update(unigram_model.ngrams)
bigram_model.context.update(unigram_model.context)

trigram_model.ngrams.update(unigram_model.ngrams)
trigram_model.ngrams.update(bigram_model.ngrams)
trigram_model.context.update(unigram_model.context)
trigram_model.context.update(bigram_model.context)

## b) Compare with the results from In Class Exercise.

In [18]:
# Sample sentences for evaluation
correct_sentence = "I want to give a speech at Ted Talk."
incorrect_sentence = "I want give a speech to at Ted Talk."

# Calculate probabilities and perplexities of each sentence
for model, name in zip([unigram_model, bigram_model, trigram_model], ["Unigram", "Bigram", "Trigram"]):
    print(f"{name} Model:")
    print(f"  Probability of correct sentence:")
    print(f"    Base on Laplace smooth = {model.sentence_probability(correct_sentence)[0]}")
    print(f"    Base on Stupid backoff = {model.sentence_probability(correct_sentence)[1]}")
    print(f"  Perplexity of correct sentence:")
    print(f"    Base on Laplace smooth = {model.sentence_perplexity(correct_sentence)[0]}")
    print(f"    Base on Stupid backoff = {model.sentence_perplexity(correct_sentence)[1]}")

    print(f"  Probability of incorrect sentence:")
    print(f"    Base on Laplace smooth = {model.sentence_probability(incorrect_sentence)[0]}")
    print(f"    Base on Stupid backoff = {model.sentence_probability(incorrect_sentence)[1]}")
    print(f"  Perplexity of incorrect sentence:")
    print(f"    Base on Laplace smooth = {model.sentence_perplexity(incorrect_sentence)[0]}")
    print(f"    Base on Stupid backoff = {model.sentence_perplexity(incorrect_sentence)[1]}")

    print(100*'-')

Unigram Model:
  Probability of correct sentence:
    Base on Laplace smooth = 2.493751526229549e-25
    Base on Stupid backoff = 2.700615182081296e-25
  Perplexity of correct sentence:
    Base on Laplace smooth = 541.6053929661736
    Base on Stupid backoff = 536.8308646005801
  Probability of incorrect sentence:
    Base on Laplace smooth = 2.493751526229549e-25
    Base on Stupid backoff = 2.7006151820812953e-25
  Perplexity of incorrect sentence:
    Base on Laplace smooth = 541.6053929661736
    Base on Stupid backoff = 536.8308646005802
----------------------------------------------------------------------------------------------------
Bigram Model:
  Probability of correct sentence:
    Base on Laplace smooth = 5.595492433883201e-25
    Base on Stupid backoff = 8.607043216510398e-17
  Perplexity of correct sentence:
    Base on Laplace smooth = 160.2293133509159
    Base on Stupid backoff = 28.871398401479873
  Probability of incorrect sentence:
    Base on Laplace smooth = 2.3

### Analyze the results
From the result, we can conclude that the Stupid backoff method performs better than Laplace smoothing method. The reason is that the Laplace smoothing treat all ngram the same way by adding 1 to the count of ngram, it leading to reduce mass probability when encounter unseen ngrams, while the Stuipid backoff consider backoff to the lower n-gram when the count of higher one is zero


## c) Use the newly built model to generate the next words for a given word sequence.

In [19]:
intial_sentence = 'I want to make'
trigram_complete_sentence = trigram_model.generate_sentence(intial_sentence,20)

Initial Sequence: i want to make
Generating...

Given Sequence: to make
Candidate Words and Probabilities:

Best Next Word: a (Probability: 0.13801244902339557)
Current Sentence: i want to make a
Given Sequence: make a
Candidate Words and Probabilities:

Best Next Word: difference (Probability: 0.12779973649538867)
Current Sentence: i want to make a difference
Given Sequence: a difference
Candidate Words and Probabilities:

Best Next Word: </s> (Probability: 0.3743016759776536)

Generated Sentence: i want to make a difference


## d) Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position. (from difflib import get_close_matches)

In [20]:
misspelled_sentence = 'I waant to mnake'
trigram_complete_sentence = trigram_model.generate_sentence_for_misspelled(misspelled_sentence)

Initial Sequence: i want to make
Generating...

Given Sequence: to make
Candidate Words and Probabilities:

Best Next Word: a (Probability: 0.13801244902339557)
Current Sentence: i want to make a
Given Sequence: make a
Candidate Words and Probabilities:

Best Next Word: difference (Probability: 0.12779973649538867)
Current Sentence: i want to make a difference
Given Sequence: a difference
Candidate Words and Probabilities:

Best Next Word: </s> (Probability: 0.3743016759776536)

Generated Sentence: i want to make a difference
